### Lets apply and use what we have learned on the blog project (My_website)!

Here are three pictures demonstrating what have in our project.

<img src='images/11-blog1.png' width='100%'>
<img src='images/12-blog2.png' width='100%'>
<img src='images/13-blog3.png' width='100%'>

If you remember we used some dummy (hardcoded) data for our posts.<br>
Those data are stored in a list called all_posts inside our views.py file.<br>
What we want to do here is to store those information in a database! and then used them to in our project!<br>

Don't forget to create superuser!<br>

You should be able to do the mentioned task! so try to it by yoursef first, and if you couldn't you can check the steps I provided in the following blocks.

<br><br><br>

1- Your models.py should look like the following block:

In [ ]:
from typing import Iterable
from django.db import models
from django.utils.text import slugify
from django.core.validators import MinLengthValidator

class Tag(models.Model):
    caption = models.CharField(max_length=20)

    def __str__(self):
        return self.caption

    class Meta:
        verbose_name_plural = 'All Tags'

class Author(models.Model):
    first_name = models.CharField(max_length=100)
    last_name = models.CharField(max_length=100)
    email = models.EmailField()


    def __str__(self):
        return f"{self.first_name} {self.last_name}"


    class Meta:
        verbose_name_plural = 'All Authors'


class Post(models.Model):
    title = models.CharField(max_length=200, null=False)
    slug = models.SlugField(unique=True, db_index=True)
    date = models.DateField(auto_now=True)
    image_name = models.CharField(max_length=200, null=True)
    excerpt = models.CharField(max_length=300)
    content = models.TextField(validators=[MinLengthValidator(10)])
    author = models.ForeignKey(Author, null=True, on_delete=models.SET_NULL, related_name='posts')
    tags = models.ManyToManyField(Tag)


    def save(self, *args, **kwargs):
        self.slug = slugify(self.title)
        super().save(*args, **kwargs)


    class Meta:
        verbose_name_plural = 'All Posts'

2- Your admin.py should look like the following block:

In [ ]:
from django.contrib import admin
from .models import Author, Post, Tag


class TagAdmin(admin.ModelAdmin):
    list_display = ['caption']

class AuthorAdmin(admin.ModelAdmin):
    list_display = ['first_name', 'last_name', 'email']

class PostAdmin(admin.ModelAdmin):
    prepopulated_fields = {'slug':('title',)}
    list_display = ['title', 'date', 'author']
    list_filter = ('author', 'date', 'tags')


admin.site.register(Tag, TagAdmin)
admin.site.register(Author, AuthorAdmin)
admin.site.register(Post, PostAdmin)

3- Run the makemigrations and migrate command (if you dont have db.sqlite3 file, create it).

In [ ]:
python manage.py makemigrations
python manage.py migrate

4- Create a Supter-User

In [ ]:
python manage.py createsuperuser

5- Run delevelopment server, and then request admin page.<br>
6- Inside the admin page, check if everything is working fine.<br>
7- Add a number of authors using the admin page.<br>
8- Add a few tags in the tag model using the admin page.<br>
9- Use the dummy (hard coded) data you have in the views.pt file and add some posts to the post model using the admin page.

### Lets work on the vies.py and templates.

1- Delete the all posts list from the views.py
2- Import data-models

### Now, Modify the view functions to send the proper data to the templates.
1- Lets start with index function:

In [ ]:
def index(request):
    latest_posts = Post.objects.all().order_by('-date')[:3]
    return render(request, "blog/index.html", {
        "posts":latest_posts
    })

You might think the code that we wrote here is inefficient, because we are loading all the posts from the database, order them and then use slicing to get the first three.<br>
You are wrong ofc. Django is smart,  It converts our code into an optimized SQL query, fetching only the necessary data – in this case, the first three rows. This approach ensures that our application remains fast and scalable, even with large datasets.

If you save everything and rty to acceess 127.0.0.1:8000 you will get an error! can you guess why?<br>

Let me explain it:
Open you models.py file and check the attributes name of your Post mode. Now Navigate to your post.html file (inside includes) and check the key names that you used. Do they match?<br>
Fix them.

You post.html file should look like the following block:

{% load static %}

<li>
    <article class='post'>
        <a href=" {% url "blog-post-detail" post.slug %}">
            <img src="{% static "blog/images/"|add:post.image_name %}" alt="{{ post.title }}">
            <div class="post__content">
                <h3>{{ post.title }}</h3>
                <p>{{ post.excerpt }}</p>
            </div>
        </a>
    </article>
</li>

2- Fix all the other view functions and heir corresponding template.

Your views.py file should look like the following block:

In [ ]:
from django.shortcuts import render
from .models import Tag, Author, Post

def index(request):
    latest_posts = Post.objects.all().order_by('-date')[:3]
    return render(request, "blog/index.html", {
        "posts":latest_posts
    })

def posts(request):
    all_posts = Post.objects.all().order_by('-date')
    return render(request, "blog/all-posts.html", {
        "all_posts": all_posts
    })

def post_detail(request, slug):
    post = Post.objects.get(slug=slug)
    return render(request, "blog/post-detail.html", {
        "post":post
    }) 

#### Fix the 404 error aswell.

In [ ]:
from django.shortcuts import render, get_object_or_404
from .models import Tag, Author, Post

def index(request):
    latest_posts = Post.objects.all().order_by('-date')[:3]
    return render(request, "blog/index.html", {
        "posts":latest_posts
    })

def posts(request):
    all_posts = Post.objects.all().order_by('-date')
    return render(request, "blog/all-posts.html", {
        "all_posts": all_posts
    })

def post_detail(request, slug):
    post = get_object_or_404(Post, slug=slug)
    return render(request, "blog/post-detail.html", {
        "post":post
    }) 

#### In the next notebook we are going to add more details to our blog project.